# Librerías

In [1]:
import time
import re
import numpy as np
import pandas as pd
from sentence_transformers import SentenceTransformer
import warnings 
warnings.filterwarnings('ignore')

c:\Users\epile\anaconda3\Lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


# Funciones

In [2]:
def sumar_numeros_rango(df, columna_texto):
    def extraer_y_sumar(text):
        numeros = re.findall(r'\d+(?:\.\d{1,2})?', text)
        numeros_en_rango = [float(num) for num in numeros if 50 <= float(num) <= 50000]
        return sum(numeros_en_rango) if numeros_en_rango else np.nan
    df['INGRESO'] = df[columna_texto].apply(lambda x: extraer_y_sumar(str(x)))
    
def get_embedding(texts):
    EMBEDDING_MODEL = "all-MiniLM-L6-v2"
    model = SentenceTransformer(EMBEDDING_MODEL)
    embedding = model.encode(texts, convert_to_numpy=True) 
    return embedding 

def cosine_similarity(vec1, vec2):
    dot_product = np.dot(vec1, vec2)
    norm_a = np.linalg.norm(vec1)
    norm_b = np.linalg.norm(vec2)
    return dot_product / (norm_a * norm_b)

# Limpieza de Datos

In [3]:
data = pd.read_csv('data/REPARACIONES.csv', encoding='latin1')
clientes = pd.read_csv('data/CLIENTES.csv', encoding='latin1')

sumar_numeros_rango(data, 'INFORMETALLER')
data_cleaned = data.dropna(subset=['INGRESO'])

data_cleaned["COMBINADO"] = (
    data_cleaned["FALLA"].fillna(" ") + " " +
    data_cleaned["OBSERVACIONES"].fillna(" ") + " " +
    data_cleaned["INFORMETALLER"].fillna(" ") )

data_cleaned.reset_index(inplace=True)

data_cleaned["COMBINADO"] = (
    data_cleaned["COMBINADO"]
    .str.replace(r"[^a-zA-Z\s]", "", regex=True)
    .str.lower())
data_cleaned = data_cleaned[['CLIENTE', 'COMBINADO', 'INGRESO', 'FECHATERMINADO']]
data_cleaned

,CLIENTE,COMBINADO,INGRESO,FECHATERMINADO
0,2,revision general tndr cambio del panel de cont...,1820.0,29/11/2018
1,12,incluir el editor de pdf contasea hienz inc...,150.0,06/12/2018
2,2,imprime negro tn mantenimiento iva,480.0,30/11/2018
3,3,mayusculas no escribe minusculasse nota mas ...,360.0,04/12/2017
4,3,rev gral margarita cinta tapa y cubierta rep...,360.0,04/12/2017
...,...,...,...,...
9370,1216,se traba las teclas y el carro coseguir cintas...,880.0,NaN
9371,1849,se traban las hojas resorte de sensor de pap...,3695.0,NaN
9372,4,pinta una ralla en el escaner mantenimiento ...,905.0,NaN
9373,882,no toma las hojas requiere de cambio de goma...,2120.0,NaN


# Creación de Embeddings

In [4]:
keywords = {
    'Impresora Laser': ['impresora laser', 'tn', 'toner', 'drum', 'tndr', 'dr', 'dl', 'fusor', 'pick up', 'pick', 'up',
                        'calor', 'presion', 'pad', 'termistor', 'toners', 'electroiman', 'lampara'],
    'Impresora Tinta': ['impresora tinta', 'reset', 'almohadillas', 'bateria', 'encoder', 'tinta',
                        '12v', 'cinta', 'perilla', 'cabezal', 'manguera', 'mascarilla'],
    'Sistemas': ['sistema', 'office', 'pantalla', 'windows', 'bisagra', 'teclado', 
                 'mouse', 'mousepad', 'ssd', 'operativo', 'disco duro', 'mayusculas', 'minusculas',
                 'ram', 'procesador', 'madre', 'correo', 'gabinete', 'bios', 'bocina'],
    'Otros': ['otros', 'margarita', 'myler', 'mylar', 'videocasetera', 'corrector']
}

In [5]:
inicio = time.time()
################################################################

imp_laser = []
imp_tinta = []
sistemas = []
otros = []

for palabra in keywords['Impresora Laser']:
    embedding = get_embedding(palabra)
    imp_laser.append(embedding)

for palabra in keywords['Impresora Tinta']:
    embedding = get_embedding(palabra)
    imp_tinta.append(embedding)

for palabra in keywords['Sistemas']:
    embedding = get_embedding(palabra)
    sistemas.append(embedding)

for palabra in keywords['Otros']:
    embedding = get_embedding(palabra)
    otros.append(embedding)


keywords_embedding = {
    'Impresora Laser':imp_laser,
    'Impresora Tinta':imp_tinta,
    'Sistemas':sistemas,
    'Otros':otros
} 

################################################################
fin = time.time()
tiempo_total = fin - inicio
print(f"El script tardó {tiempo_total:.4f} segundos en ejecutarse.")

El script tardó 51.7316 segundos en ejecutarse.


In [6]:
inicio = time.time()
################################################################

muestra_embedding = []
muestra = data_cleaned['COMBINADO'].reset_index()

for i in range(len(muestra)):
    embedding = get_embedding(muestra['COMBINADO'][i])
    muestra_embedding.append(embedding)


################################################################
fin = time.time()
tiempo_total = fin - inicio
print(f"El script tardó {tiempo_total:.4f} segundos en ejecutarse.")

El script tardó 8492.5280 segundos en ejecutarse.


In [7]:
resultados_correlacion = []

for i, vector in enumerate(muestra_embedding):
    for key, lista_vectores in keywords_embedding.items():
        for j, vector_dict in enumerate(lista_vectores):
            cos_sim = cosine_similarity(vector, vector_dict)
            resultados_correlacion.append({
                "Número de falla": i + 1, 
                "Segmento": key,  
                "Palabra": keywords[key][j], 
                "Similitud Coseno": cos_sim
            })

df_resultados = pd.DataFrame(resultados_correlacion)

# Similitud Coseno por falla

In [8]:
idx = df_resultados.groupby('Número de falla')['Similitud Coseno'].idxmax()
fallas = df_resultados.loc[idx].reset_index(drop=True)
fallas

,Número de falla,Segmento,Palabra,Similitud Coseno
0,1,Impresora Laser,tndr,0.298307
1,2,Sistemas,procesador,0.315053
2,3,Impresora Tinta,impresora tinta,0.391361
3,4,Sistemas,sistema,0.408414
4,5,Sistemas,teclado,0.479646
...,...,...,...,...
9370,9371,Sistemas,teclado,0.566165
9371,9372,Impresora Tinta,encoder,0.344659
9372,9373,Impresora Laser,drum,0.381613
9373,9374,Impresora Tinta,impresora tinta,0.296538


# Clusterización

In [9]:
prueba = data_cleaned
prueba['CLUSTER'] = fallas['Segmento']
prueba

,CLIENTE,COMBINADO,INGRESO,FECHATERMINADO,CLUSTER
0,2,revision general tndr cambio del panel de cont...,1820.0,29/11/2018,Impresora Laser
1,12,incluir el editor de pdf contasea hienz inc...,150.0,06/12/2018,Sistemas
2,2,imprime negro tn mantenimiento iva,480.0,30/11/2018,Impresora Tinta
3,3,mayusculas no escribe minusculasse nota mas ...,360.0,04/12/2017,Sistemas
4,3,rev gral margarita cinta tapa y cubierta rep...,360.0,04/12/2017,Sistemas
...,...,...,...,...,...
9370,1216,se traba las teclas y el carro coseguir cintas...,880.0,NaN,Sistemas
9371,1849,se traban las hojas resorte de sensor de pap...,3695.0,NaN,Impresora Tinta
9372,4,pinta una ralla en el escaner mantenimiento ...,905.0,NaN,Impresora Laser
9373,882,no toma las hojas requiere de cambio de goma...,2120.0,NaN,Impresora Tinta


In [10]:
prueba['SIMILITUD COSENO'] = fallas['Similitud Coseno']
prueba

,CLIENTE,COMBINADO,INGRESO,FECHATERMINADO,CLUSTER,SIMILITUD COSENO
0,2,revision general tndr cambio del panel de cont...,1820.0,29/11/2018,Impresora Laser,0.298307
1,12,incluir el editor de pdf contasea hienz inc...,150.0,06/12/2018,Sistemas,0.315053
2,2,imprime negro tn mantenimiento iva,480.0,30/11/2018,Impresora Tinta,0.391361
3,3,mayusculas no escribe minusculasse nota mas ...,360.0,04/12/2017,Sistemas,0.408414
4,3,rev gral margarita cinta tapa y cubierta rep...,360.0,04/12/2017,Sistemas,0.479646
...,...,...,...,...,...,...
9370,1216,se traba las teclas y el carro coseguir cintas...,880.0,NaN,Sistemas,0.566165
9371,1849,se traban las hojas resorte de sensor de pap...,3695.0,NaN,Impresora Tinta,0.344659
9372,4,pinta una ralla en el escaner mantenimiento ...,905.0,NaN,Impresora Laser,0.381613
9373,882,no toma las hojas requiere de cambio de goma...,2120.0,NaN,Impresora Tinta,0.296538


In [11]:
clientes = clientes[['CLIENTE', 'COHORT']]
clientes

,CLIENTE,COHORT
0,0,21/11/2014
1,2,22/11/2017
2,3,22/11/2017
3,4,07/06/2024
4,5,27/04/2018
...,...,...
1849,1850,09/10/2024
1850,1851,11/10/2024
1851,1852,16/10/2024
1852,1853,17/10/2024


In [17]:
df_merged = pd.merge(prueba, clientes, on='CLIENTE', how='inner')
df_merged['COHORT'] = pd.to_datetime(df_merged['COHORT'])
df_merged = df_merged.dropna(subset='FECHATERMINADO')
df_merged

,CLIENTE,COMBINADO,INGRESO,FECHATERMINADO,CLUSTER,SIMILITUD COSENO,COHORT
0,2,revision general tndr cambio del panel de cont...,1820.0,29/11/2018,Impresora Laser,0.298307,2017-11-22
1,2,imprime negro tn mantenimiento iva,480.0,30/11/2018,Impresora Tinta,0.391361,2017-11-22
2,2,rev gral cartucho toner reparacion traccion de...,1030.0,02/01/2019,Impresora Laser,0.508475,2017-11-22
3,2,rev gral sin toner con cable de energia bue...,530.0,29/11/2018,Impresora Laser,0.493519,2017-11-22
4,2,rev gral sin toner cambio de contacto del ca...,480.0,29/11/2018,Impresora Laser,0.448675,2017-11-22
...,...,...,...,...,...,...,...
9367,1835,en el area del impime obscuro en doble cara ta...,2815.0,20/09/2024,Sistemas,0.325016,2024-09-13
9368,1837,no escanea no saca copias marca papel atascado...,3532.0,30/09/2024,Sistemas,0.275756,2024-09-13
9371,1843,marcaba error de fusor y luego error de escane...,4790.0,30/09/2024,Impresora Laser,0.458887,2024-09-24
9372,1841,arruga las hojas y se atora cambio de filmin...,2575.0,26/09/2024,Otros,0.408079,2024-09-20


# Exportar a CSV

In [18]:
df_merged = df_merged.rename(columns={'COMBINADO': 'DESCRIPCION FALLA',
                                      'FECHATERMINADO': 'FECHA REPARACION'})
df_merged.to_csv('reparaciones.csv', index=False)